In [1]:
from document_intelligence_functions import DocumentIntelligence
import os
import pandas as pd

In [2]:
def process_table(table):
    data = {}
    for index, row in table.iterrows():
        # Concatenar todos los elementos no nulos de la fila en una sola cadena
        full_row = ' '.join(str(item) for item in row if pd.notna(item))
        # Dividir la cadena completa en posibles pares clave-valor
        parts = full_row.split(':')
        current_key = None
        
        # Iterar sobre las partes para formar correctamente los pares clave-valor
        for i, part in enumerate(parts):
            if i == 0:
                # El primer elemento siempre es una clave
                current_key = part.strip()
            else:
                # Los siguientes elementos pueden ser valor de la clave anterior y clave del siguiente par
                # Dividir en el último espacio para separar el valor de la nueva clave
                last_space_index = part.rfind(' ')
                if last_space_index != -1:
                    value = part[:last_space_index].strip()
                    new_key = part[last_space_index:].strip()
                    
                    if current_key and value:
                        data[current_key] = value
                    current_key = new_key
                else:
                    # En caso de que no haya espacio, toda la parte es un valor
                    if current_key:
                        data[current_key] = part.strip()
                        
        # Asegurarse de añadir el último par si es necesario
        if current_key and i == len(parts) - 1 and parts[i]:
            data[current_key] = parts[i].strip()
    
    return data


In [4]:
# Cambiar al directorio deseado (si es necesario)
# os.chdir('/path/to/your/directory')

# Comprobar el directorio actual para confirmar que es correcto
print("Directorio de trabajo actual:", os.getcwd())

# Ruta al documento PDF
file_path = "F6CR_Aprobada_Hamaca-100D_27052022.pdf"

Directorio de trabajo actual: c:\Users\OlmeiroOrozcoOrtiz\Downloads\OCR Frontera


In [5]:
# Crear una instancia de la clase DocumentIntelligence
doc_intelligence = DocumentIntelligence()

# Intentar llamar al método analyze_read para procesar el documento
try:
    text_content, tables = doc_intelligence.analyze_read(file_path=file_path)
except Exception as e:
    print(f"Error al procesar el documento: {e}")
    exit()


Error al procesar el documento: No connection adapters were found for '/formrecognizer/documentModels/prebuilt-layout:analyze?stringIndexType=unicodeCodePoint&api-version=2023-07-31'


: 

In [ ]:
# Imprimir el contenido del documento
# for page_text in text_content:
#     print("Contenido de la página:", page_text)

In [ ]:
# Preparar un DataFrame para guardar todos los datos
all_data = []

# Si hay tablas, procesarlas
if tables:
    for table in tables:
        data = process_table(table)
        if data:
            print(data)  # Opcional: imprimir para depuración
            all_data.append(data)

In [ ]:
# Convertir todos los datos en un DataFrame y guardarlos en Excel
if all_data:
    df = pd.DataFrame(all_data)
    try:
        df.to_excel("datos_pozo.xlsx", index=False)
        print("Todos los datos han sido exportados a Excel exitosamente.")
    except Exception as e:
        print(f"Error al guardar en Excel: {e}")
else:
    print("No se encontraron datos para exportar.")
